In [3]:
import gymnasium as gym
from gymnasium.utils import play
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
import keras
import numpy as np

In [4]:
env = gym.make('ALE/Boxing-v5', render_mode = 'rgb_array',obs_type='grayscale')

In [8]:
# Utility functions

def preprocess(obs):
    return obs[30:180, 30:130]


In [35]:
# Mass generation of training data using tree search getting all possible rewards from a given state

def generate_data(env, num_episodes, max_steps_per_episode, preprocess = lambda x: x):
    for i in range(num_episodes):
        print(f"Episode {i+1}/{num_episodes}")
        obs = env.reset()
        obs = preprocess(obs[0])
        for j in range(max_steps_per_episode):
            r_matrix = np.zeros(env.action_space.n)
            for action in range(env.action_space.n):
                env_copy = env.unwrapped.clone_state(include_rng = False)
                step = env.step(action)
                next_obs = step[0]
                reward = step[1]
                done = step[2]
                next_obs = preprocess(next_obs)
                r_matrix[action] = reward
                env.unwrapped.restore_state(env_copy)

            #Appends the data to the training set in a csv file
            with open('training_data.csv', 'a') as f:
                np.savetxt(f, np.append(obs.flatten(), r_matrix), delimiter = ',')
            if done:
                break
            

            if np.random.rand() < 0.5:
                action = env.action_space.sample()
            else:
                action = np.argmax(r_matrix)

            step = env.step(action)
            obs = step[0]
            reward = step[1]
            done = step[2]
            obs = preprocess(obs)

            if done:
                break


In [36]:
generate_data(env, 10, 1000, preprocess)

Episode 1/10


KeyboardInterrupt: 